In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
%cd /content/drive/MyDrive/Sentiment_Analysis

/content/drive/MyDrive/Sentiment_Analysis


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from matplotlib import pyplot as plt
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import unicodedata

In [ ]:
! sudo apt install openjdk-8-jdk
! sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
! pip install language-check
! pip install pycontractions

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-8-jdk is already the newest version (8u402-ga-2ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached language-check-1.1.tar.gz (33 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for language-check
  Running setup.py clean for language-check
Failed to build language-check
ERROR: Could not build wheels for language-check, which is required to install pyproject.toml-based projects
  Using cached pycontractions-2.0.1-py3-none-any.whl (9.6 kB)
  Using cached language-check-1.1.tar.gz (33 kB)
  Preparing metadata (setup.py) ... done
  Using cached pyemd-1.0.0-cp310-cp310-manylinux_2_17_x86_

In [64]:
pip install autocorrect

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [121]:
train_df = pd.read_csv("data/train.csv")
print(train_df.shape)
train_df.head()

(27481, 4)


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [120]:
test_df = pd.read_csv("data/test.csv")
print(test_df.shape)
test_df.head()

(3534, 3)


,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [123]:
def clean_text_data(data: pd.Series) -> pd.Series:
  url = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
  assert data.isna().sum() == 0, data[data.isna()]
  data = data.map(lambda x:str(x).lower()) #lower case
  data = data.map(lambda x:re.sub(r"\b[^\s]+@[^\s]+[.][^\s]+\b", "", x)) #email
  data = data.map(lambda x:re.sub(url, "", x)) #url
  data = data.map(lambda x:re.sub(r'[^a-zA-z.,!?/:;\"\'\s]', "", x)) #numbers
  data = data.map(lambda x:re.sub(r'^\s*|\s\s*', ' ', x).strip()) #white space
  data = data.map(lambda x:''.join([c for c in x if c not in string.punctuation])) #punctuations
  data = data.map(lambda x:re.sub(r'[^a-zA-z0-9.,!?/:;\"\'\s]', '', x)) #special char
  data = data.map(lambda x:unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')) #unicode
  tokenizer = RegexpTokenizer(r'\w+')
  data = data.map(lambda x:tokenizer.tokenize(x))  #remove punctuation and tokenize
  return data

train_df.dropna(inplace=True)
train_df.text = clean_text_data(train_df.text)
train_df.selected_text = clean_text_data(train_df.selected_text)
test_df.text = clean_text_data(test_df.text)

In [124]:
train_df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"[id, have, responded, if, i, were, going]","[id, have, responded, if, i, were, going]",neutral
1,549e992a42,"[sooo, sad, i, will, miss, you, here, in, san,...","[sooo, sad]",negative
2,088c60f138,"[my, boss, is, bullying, me]","[bullying, me]",negative
3,9642c003ef,"[what, interview, leave, me, alone]","[leave, me, alone]",negative
4,358bd9e861,"[sons, of, why, couldnt, they, put, them, on, ...","[sons, of]",negative


In [125]:
test_df.head()

,textID,text,sentiment
0,f87dea47db,"[last, session, of, the, day]",neutral
1,96d74cb729,"[shanghai, is, also, really, exciting, precise...",positive
2,eee518ae67,"[recession, hit, veronique, branquinho, she, h...",negative
3,01082688c6,"[happy, bday]",positive
4,33987a8ee5,"[i, like, it]",positive


In [112]:
# #to use spellcheck
# from autocorrect import Speller
# spell = Speller(lang="en")
# df.input = df.input.map(lambda x:tokenizer.tokenize(x))  #remove punctuation and tokenize
# df.input = df.input.map(lambda x:[spell(i) for i in x])  #spell check

In [126]:
train_df.to_csv("data/preprocessed_train.csv",index=False)
test_df.to_csv("data/preprocessed_test.csv",index=False)